In [ ]:
# default_exp exposure
%load_ext autoreload
%autoreload 2

# Exposure
> calculate exposure for a point source from a GTI and FT2 files

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from astropy.io import fits
import numpy as np
import pandas as pd
from scipy.integrate import simps
from light_curves.config import MJD

In [ ]:
#export
def _process_ft2(config, source, ft2_file_path, gti, effective_area):
    """Process a set of FT2 files, with S/C history data
    Parameters:
        - config -- verbose, cos_theta_max, z_max
        - source -- l,b for position
        - ft2_files -- list of spacecraft files
        - gti -- GTI object with allowed intervals
        - effective_area -- function of energy and angle with respect to zenity
    Generate a dataset with fields:
        - start, stop : start  and stop times in MJD
        - exposure    : calculated exposure using effective area

     """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    ft2_files = list(ft2_file_path.glob('*.fits'))
    if config.verbose>1:
        print(f'Processing {len(ft2_files)} S/C history (FT2) files')
        print(f'  applying cuts cos(theta) < {config.cos_theta_max},  z < {config.z_max}')
    sc_data=[]
    for filename in ft2_files:
        with fits.open(filename) as hdu:
            scdata = hdu['SC_DATA'].data
            # get times to check against MJD limits and GTI
            start, stop = [MJD(np.array(scdata.START, float)),
                           MJD(np.array(scdata.STOP, float))]
            if config.mjd_range is not None:
                a,b=  config.mjd_range
                if stop[-1]<a:
                    print(f'\tfile {filename}: skip, before selected range' )
                    continue
                elif start[0]>b:
                    print(f'\tfile {filename}: quit, beyond range')
                    break
            # apply GTI to bin center (avoid edge effects?)
            in_gti = gti(0.5*(start+stop))
            if config.verbose>2:
                print(f'\tfile {filename}: {len(start)} entries, {sum(in_gti)} in GTI')
            t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
                [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]
            sc_data.append( pd.DataFrame(dict(t) ) )
    df = pd.concat(sc_data, ignore_index=True)

    # calculate cosines with respect to sky direction
    sc = source
    ra_r,dec_r = np.radians(sc.ra), np.radians(sc.dec)
    sdec, cdec = np.sin(dec_r), np.cos(dec_r)

    def cosines( ra2, dec2):
        ra2_r =  np.radians(ra2.values)
        dec2_r = np.radians(dec2.values)
        return np.cos(dec2_r)*cdec*np.cos(ra_r-ra2_r) + np.sin(dec2_r)*sdec

    pcosines = cosines(df.ra_scz,    df.dec_scz)
    zcosines = cosines(df.ra_zenith, df.dec_zenith)

    # mask out entries too close to zenith, or too far away from ROI center
    mask =   (pcosines >= config.cos_theta_max) & (zcosines>=np.cos(np.radians(config.z_max)))
    if config.verbose>1:
        print(f'\tFound {len(mask):,} S/C entries:  {sum(mask):,} remain after zenith and theta cuts')
    dfm = df.loc[mask,:]
    livetime = dfm.livetime.values
    config.dfm = dfm ##############debug
    # apply MJD range if present. note times in MJD
    start, stop = dfm.start,dfm.stop
    lims = slice(None)
    if config.mjd_range is not None:
#         a, b = config._get_limits(start)
        a, b = np.searchsorted(start, config.mjd_range)
        if a>0 or b<len(start):
            if config.verbose>1:
                print(f'\tcut from {len(start):,} to {a} - {b}, or {b-a:,} entries after MJD range selection')
            dfm = dfm.iloc[a:b]
            lims = slice(a,b)


    expose = _exposure(config, effective_area, livetime[lims], pcosines[mask][lims])
    return pd.DataFrame(dict(start=start[lims],stop=stop[lims], exposure=expose))

In [ ]:
#export
def _exposure(config, effective_area, livetime, pcosine):
    """return exposure calculated for each pair in livetime and cosines arrays

    uses effective area
    """
    assert len(livetime)==len(pcosine), 'expect equal-length arrays'

    # get a set of energies and associated weights from a trial spectrum

    emin,emax = config.energy_range
    loge1=np.log10(emin); loge2=np.log10(emax)

    edom=np.logspace(loge1, loge2, int((loge2-loge1)*config.bins_per_decade+1))
    if config.verbose>1:
        print(f'Calculate exposure using the energy domain'\
              f' {emin}-{emax} {config.bins_per_decade} bins/decade' )
    base_spectrum = eval(config.base_spectrum) #lambda E: (E/1000)**-2.1
    assert base_spectrum(1000)==1.
    wts = base_spectrum(edom)

    # effectivee area function from
    ea = effective_area

    # a table of the weighted for each pair in livetime and pcosine arrays
    rvals = np.empty([len(wts),len(pcosine)])
    for i,(en,wt) in enumerate(zip(edom,wts)):
        faeff,baeff = ea([en],pcosine)
        rvals[i] = (faeff+baeff)*wt

    aeff = simps(rvals,edom,axis=0)/simps(wts,edom)
    return (aeff*livetime)

In [ ]:
#export
def get_exposure(config, files, gti, source, use_cache=True):
    """Return the exposure for the source
    If gti is None, regenerate it
    """
    from  light_curves.load_gti import get_gti
    from light_curves.effective_area import EffectiveArea
    
    fcache = files.cache/f'{source.filename}_exposure.pkl' if use_cache else None

    if fcache and fcache.exists():    
        if config.verbose>1:
            print(f'restoring exposure from {fcache} ' , end='')
        exposure = pd.read_pickle(fcache)
        if config.verbose>1:
            print(f'{len(exposure)} entries, MJD {exposure.iloc[0].start:.0f}'
                  f' - {exposure.iloc[-1].stop:.0f}')
            
    else:
        gti = gti or get_gti(config, files.gti)
        aeff = EffectiveArea(file_path = files.aeff)
        exposure =  _process_ft2(config, source, files.ft2, gti, aeff)

        if fcache:
            if config.verbose>1:
                print(f'saving exposure to {fcache}')
            exposure.to_pickle(fcache)
    return exposure

In [ ]:
from light_curves.config import Config, Files, PointSource
files = Files()
config=Config()
source = PointSource('Geminga')

exposure = get_exposure(config, files, None, source)
    

Processing 11 FITS files with GTI information ...  11 files, 63635 intervals with 3,322 days live time
	GTI MJD range: 54682.66-58698.08, good fraction 0.83 
Processing 12 S/C history (FT2) files
  applying cuts cos(theta) < 0.4,  z < 100
	file /home/burnett/work/lat-data/ft2/ft2_2008.fits: 362996 entries, 360944 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2009.fits: 874661 entries, 870446 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2010.fits: 889547 entries, 884697 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2011.fits: 882832 entries, 871672 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2012.fits: 881317 entries, 868109 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2013.fits: 885307 entries, 867342 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2014.fits: 894730 entries, 886570 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2015.fits: 890006 entries, 886086 in GTI
	file /home/burnett/work/lat-data/ft2/ft2_2016.fits: 890933 entries, 884823 in GTI
	file /home/bu

The result is a tuple of time intervals and the exposure

In [ ]:
if exposure is not None:
    print(exposure)

                start          stop       exposure
0        54682.656038  54682.656375  112998.052089
1        54682.656375  54682.656722  116901.160578
2        54682.656722  54682.657069  116857.143435
3        54682.657069  54682.657416  116791.434285
4        54682.657416  54682.657764  116876.550059
...               ...           ...            ...
9609753  58698.052602  58698.052949   52427.727986
9609754  58698.052949  58698.053296   49343.687807
9609755  58698.053296  58698.053643   44567.360241
9609756  58698.053643  58698.053991   38779.120217
9609757  58698.053991  58698.054338   31786.491045

[2695715 rows x 3 columns]


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date